In [ ]:
# 匯入套件
import requests as req
from bs4 import BeautifulSoup as bs
import re, json
from time import sleep
from random import randint

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent(cache=True) # cache=True 表示從已經儲存的列表中提取

# 設定請求標頭
my_headers = {
    'user-agent': ua.random
}

# 設定 cookie
my_cookies = {
    'age_checked_for': "12190"
}

# 整理所有取得資料的變數
listData = []

# 欲抓取資料的網址
domainName = 'https://www.zeczec.com/'

# 指定最後頁數
pages = 1

# 取得首頁列表資訊
def getMainData():
    for page in range(1, pages + 1):
        # 取得回應
        res = req.get(url = f'{domainName}categories?page={page}', headers = my_headers)

        # 初始化 soup 物件
        soup = bs(res.text, 'lxml')

        # 取得所有超連結
        for a in soup.select('div.flex.gutter3-l a.db'):
            # 取得圖片連結
            strStyle = a.select_one('div.aspect-ratio-project-cover')['data-bg']
            regexImg = r"https:\/\/assets\.zeczec\.com\/asset_\d+_image_big\.(jpe?g|png)"
            matchImg = re.match(regexImg, strStyle)
            strImg = matchImg[0]

            # 取得超連結
            strLink = domainName + a['href']

            # 取得標題文字
            strTitle = a.select_one('h3.b').get_text()

            # 整理首頁資料
            listData.append({
                "cover": strImg,
                "link": strLink,
                "title": strTitle
            })

# 取得詳細頁面資訊
def getDetailData():
    # 走訪每一個頁面
    for index, _dict in enumerate(listData):        
        # 輸出網址，以便於 debug
        print(_dict['link'])
        
        # 取得回應
        res = req.get(url = _dict['link'], headers = my_headers, cookies = my_cookies)

        # 初始化 soup 物件
        soup = bs(res.text, 'lxml')
        
        # 取得價格
        strPrice = soup.select_one('div.js-sum-raised').get_text()
        regexPrice = r"\D"
        strPrice = re.sub(regexPrice, "", strPrice)
        
        # 取得贊助人數
        strBacker = soup.select_one('span.js-backers-count').get_text()
        
        # 取得剩餘時間
        strTime = "longterm"
        if soup.select_one('span.js-time-left') != None:
            strTime = soup.select_one('span.js-time-left').get_text()
            regexTime = r"\d+"
            strTime = re.search(regexTime, strTime)[0]
            
        # 取得持續時間
        dictDuration = {"begin": "", "end": ""}
        strDuration = soup.select_one('div.mb2.f7').get_text()
        regexDuration = r"(\d{4}\/\d{2}\/\d{2}\s\d{2}:\d{2})(\s–\s(\d{4}\/\d{2}\/\d{2}\s\d{2}:\d{2}))?"
        matchDuration = re.search(regexDuration, strDuration)
        dictDuration['begin'] = matchDuration[1]
        if matchDuration[3] != None:
            dictDuration['end'] = matchDuration[3]
            
        # 整理詳細頁面資料
        listData[index]['price'] = strPrice
        listData[index]['backer'] = strBacker
        listData[index]['time'] = strTime
        listData[index]['duration'] = dictDuration
        
        # 隨機等待
        sleep(randint(5,10))

# 儲存成 json 檔案
def saveJson():
    with open("crowdfunding.json", "w", encoding="utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False, indent=4) )
        
# 主程式
if __name__ == "__main__":
    getMainData()
    getDetailData()
    saveJson()